<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/hackerearth/get_a_room_hackathon/notebooks/03_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%%capture
!pip install --upgrade xgboost
!pip install --upgrade optuna

In [ ]:
import os
import gc
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

import xgboost
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)

from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBRegressor

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
#remove cell to run future versions
assert optuna.__version__ == '2.10.1', f'Change in Optuna version. Original notebook version: 2.10.1'
assert xgboost.__version__ == '1.6.1', f'Change in XGBoost version. Original notebook version: 1.6.1'

In [ ]:
SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [ ]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    GPU = True
except Exception:
    GPU = False

print(f'GPU available: {GPU}')

GPU available: True


**Data**

In [ ]:
data_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/hackerearth/get_a_room_hackathon/data/'

train = pd.read_csv(data_url + 'processed/train.csv')
test = pd.read_csv(data_url + 'processed/test.csv')
test_index = test.pop('Property_ID')

target_url = data_url + 'raw/train.csv'
target = pd.read_csv(target_url, usecols=['Habitability_score']).iloc[:, 0]

In [ ]:
features = list(train.columns)

num_features = ['Property_Area', 'Number_of_Windows', 
                'Number_of_Doors', 'Traffic_Density_Score', 
                'Air_Quality_Index', 'Neighborhood_Review']

cat_features = [f for f in features if f not in num_features]

In [ ]:
train[cat_features] = train[cat_features].astype('category')
test[cat_features] = test[cat_features].astype('category')

[From EDA notebook](https://github.com/stiwari-ds/data-science-competitions/blob/main/analytics_vidhya/jobathon_aug22/notebooks/01_eda.ipynb)

In [ ]:
top_features = ['Neighborhood_Review', 'Furnishing', 'Crime_Rate', 'Power_Backup']

rejected_features = ['Number_of_Doors', 'Traffic_Density_Score', 'aqi_range']
reduced_features = [f for f in features if f not in rejected_features]

# Hyperparameter tuning

In [ ]:
def objective(trial, data, base_params):

    scores = []
    X, y = data
    cat_features = list(X.select_dtypes(include='category').columns)

    param_grid = {
        'learning_rate': trial.suggest_float(
            'learning_rate', 0.05, 0.4, step=0.025),
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float(
            'colsample_bytree', 0.5, 1.0, step=0.05),
        'colsample_bylevel': trial.suggest_float(
            'colsample_bylevel', 0.5, 1.0, step=0.05),
        'colsample_bynode': trial.suggest_float(
            'colsample_bynode', 0.5, 1.0, step=0.05),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'gamma': trial.suggest_float('gamma', 0, 20),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e2, log=True),
        'lambda': trial.suggest_float('lambda', 1e-5, 1e2, log=True)
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        model = XGBRegressor(
            **base_params, 
            **param_grid,
            callbacks=[XGBoostPruningCallback(
                trial=trial, 
                observation_key='validation_0-rmse')]
        )
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        preds = model.predict(X_val)
        scores.append(mean_squared_error(y_val, preds, squared=False))
    
    return np.mean(scores)

In [ ]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction)
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True)
    
    return study

# Cross-validation

In [ ]:
def evaluate_model(data, model_params, verbose=True):
    
    preds_test = []
    scores_r2 = [] #validation set r2-scores
    scores_rmse = [] #validation set RMSE loss
    
    X, X_test, y = data
    cat_features = list(X.select_dtypes(include='category').columns)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = XGBRegressor(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0)
        
        preds_val = model.predict(X_val)
        preds_test.append(model.predict(X_test))
        
        scores_rmse.append(mean_squared_error(y_val, preds_val, squared=False))
        scores_r2.append(r2_score(y_val, preds_val))
    
    if verbose:
        scores_df = pd.DataFrame.from_dict({
            'RMSE': scores_rmse,
            'R2': scores_r2
        })
        scores_df.index.name = 'Fold'
        display(scores_df)
    
    print(f'Average RMSE = {np.mean(scores_rmse):.4f} (with std = {np.std(scores_rmse):.4f})')
    print(f'Average R2-score = {np.mean(scores_r2):.4f} (with std = {np.std(scores_r2):.4f})\n')
    
    preds_test = np.mean(np.column_stack(preds_test), axis=1)
    return preds_test

In [ ]:
def run_experiment(data, n_trials=5):
        
    X, X_test, y = data
    
    base_params = {
        'objective': 'reg:squarederror',
        'n_estimators': 10000,
        'booster': 'gbtree',
        'eval_metric': 'rmse',
        'early_stopping_rounds': 50,
        'tree_method': 'gpu_hist' if GPU else 'hist', 
        'enable_categorical': GPU,
        'max_cat_to_onehot': 6, #internal one-hot encoding
        'verbosity': 1,
        'seed': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='minimize')
    print(f'Best trial: {study.best_trial.number} -> Best value (RMSE): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    preds_test = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params)
    
    return preds_test

In [ ]:
%%time
preds_all = run_experiment(
    (train[features], test[features], target), 500)

---------------Hyperparameter tuning---------------
Best trial: 405 -> Best value (RMSE): 5.93284
Best hyperparameters:
learning_rate        - 0.05
max_depth            - 12
subsample            - 0.75
colsample_bytree     - 0.9
colsample_bylevel    - 0.9
colsample_bynode     - 0.7
min_child_weight     - 5
gamma                - 15.16067712325734
alpha                - 0.2866930272108723
lambda               - 0.014793815846558643
-----------------Cross-validation------------------


,RMSE,R2
Fold,,
0,5.9183,0.8226
1,6.0023,0.8222
2,5.9535,0.8220
3,6.0182,0.8192
4,5.7718,0.8309


Average RMSE = 5.9328 (with std = 0.0880)
Average R2-score = 0.8234 (with std = 0.0039)

CPU times: user 47min 43s, sys: 13.3 s, total: 47min 56s
Wall time: 46min 32s


# Generating submissions

In [ ]:
preds_all[:5]

array([30.367  , 80.28141, 66.61123, 71.2711 , 74.77852], dtype=float32)

In [ ]:
sub_all = pd.DataFrame({
    'Property_ID': test_index,
    'Habitability_score': preds_all
})
sub_all.to_csv('03_sub_all.csv', index=False)

In [ ]:
!head 03_sub_all.csv

Property_ID,Habitability_score
0x6e93,30.367
0x8787,80.28141
0x6c17,66.61123
0x9dbd,71.2711
0xbfde,74.77852
0x6a39,70.15763
0x47a6,74.32289
0x7687,71.8647
0x963a,74.88786
